In [19]:
import random
import plotly.express as px
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

randomlist = []
for i in range(0,5000): #pulling random numbers for yield
    n = random.randint(150,275)
    randomlist.append(n)

randomlist2 = []
for i in range(0,5000): #pulling random numbers for price
    n = round(random.uniform(2.50,7),2)
    randomlist2.append(n)

data = DataFrame(randomlist,columns = ['Yield'])
data['Price'] = randomlist2

data['Revenue'] = data['Yield']*data['Price']

data

,Yield,Price,Revenue
0,250,2.65,662.50
1,161,6.14,988.54
2,177,2.71,479.67
3,171,3.53,603.63
4,160,5.81,929.60
...,...,...,...
4995,166,5.10,846.60
4996,274,6.31,1728.94
4997,156,6.39,996.84
4998,239,5.16,1233.24


In [2]:
c_data = pd.read_csv('/Users/isaacwerries/Desktop/542 Data.csv')
ci_price = pd.read_csv('/Users/isaacwerries/Desktop/CI Prices.csv')

del c_data['Program'], c_data['Week Ending'], c_data['Zip Code'], c_data['Region'], c_data['watershed_code'],c_data['Domain'], c_data['Domain Category'], c_data['Geo Level'], c_data['State ANSI'], c_data['Ag District'], c_data['Ag District Code'],c_data['Watershed']
corn = c_data[c_data['Commodity'] == 'CORN'] #splitting the dataframe based on crop
corn = corn[corn['Year'] > 2000].copy()
beans = c_data[c_data['Commodity'] == 'SOYBEANS'] #splitting the dataframe based on crop
beans = beans[beans['Year'] > 2000].copy()

In [3]:
#Linear Regression for corn
cornx = np.array(corn['Value']).reshape((-1,1))
corny = np.array(corn['Year']).reshape((-1,1))
corny.reshape((-1,1))
lr = LinearRegression()
regress = lr.fit(corny,cornx)
cinter = regress.intercept_ #intercept
cslope = regress.coef_ #slope

In [4]:
cinter = cinter[0]
cinter

-4858.711407747956

In [5]:
cslope = cslope[0]
cslope

array([2.49784023])

In [6]:
ci = pd.read_csv('/Users/isaacwerries/Desktop/CI Prices.csv')
corn = pd.merge(ci,corn)

In [24]:
corn['Trend fitted yield'] = corn['Year']*cslope+cinter
corn['Trend adjusted yield'] = corn['Value']+cslope*(2020-corn['Year'])
corn['Random Price'] = [round(random.uniform(2.50,7),2) for k in corn.index]
corn['Revenue'] = corn['Random Price']*corn['Trend adjusted yield']
corn = pd.merge(ci_price,corn)
del corn['Beans Base Price'], corn['Beans Harvest Price']
aph1 = corn.groupby(['County'])['Value'].agg(['sum','count']).reset_index()
corn = pd.merge(corn,aph1)
corn['County APH'] = ((corn['sum'])/(corn['count']))
corn['bp guarantee'] = corn['Trend adjusted yield']*corn['Corn Base Price']
corn['hp guarantee'] = corn['Trend adjusted yield']*corn['Corn Harvest Price']
corn['Revenue Guarantee 75%'] = corn[['bp guarantee','hp guarantee']].max(axis = 1)*.75
corn['Revenue Guarantee 80%'] = corn[['bp guarantee','hp guarantee']].max(axis = 1)*.8
corn['Revenue Guarantee 85%'] = corn[['bp guarantee','hp guarantee']].max(axis = 1)*.85
corn['0'] = 0
corn['75% dif'] = corn['Revenue Guarantee 75%']-corn['Revenue']
corn['80% dif'] = corn['Revenue Guarantee 80%']-corn['Revenue']
corn['85% dif'] = corn['Revenue Guarantee 85%']-corn['Revenue']
corn['75% payout'] = corn[['75% dif','0']].max(axis = 1)
corn['80% payout'] = corn[['80% dif','0']].max(axis = 1)
corn['85% payout'] = corn[['85% dif','0']].max(axis = 1)
corn['75% Revenue'] = corn['75% payout']+corn['Revenue']
corn['80% Revenue'] = corn['80% payout']+corn['Revenue']
corn['85% Revenue'] = corn['85% payout']+corn['Revenue']
corn.head()

,Year,Corn Base Price,Corn Harvest Price,Period,State,County,County ANSI,Commodity,Data Item,Value,...,0,75% dif,80% dif,85% dif,75% payout,80% payout,85% payout,75% Revenue,80% Revenue,85% Revenue
0,2001,2.46,2.05,YEAR,ILLINOIS,DE WITT,39.0,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",158.0,...,0,-167.449056,-142.177603,-116.906151,0.0,0.0,0.0,546.520845,546.520845,546.520845
1,2002,2.32,2.52,YEAR,ILLINOIS,DE WITT,39.0,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",133.0,...,0,-537.442595,-515.019493,-492.596391,0.0,0.0,0.0,873.789119,873.789119,873.789119
2,2003,2.42,2.37,YEAR,ILLINOIS,DE WITT,39.0,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",181.0,...,0,-667.037902,-639.998845,-612.959788,0.0,0.0,0.0,1072.623762,1072.623762,1072.623762
3,2004,2.83,1.99,YEAR,ILLINOIS,DE WITT,39.0,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",181.0,...,0,-538.603269,-507.336659,-476.070048,0.0,0.0,0.0,1007.602423,1007.602423,1007.602423
4,2005,2.32,1.93,YEAR,ILLINOIS,DE WITT,39.0,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",162.0,...,0,-686.168556,-663.030314,-639.892072,0.0,0.0,0.0,1033.242186,1033.242186,1033.242186


In [25]:
#Linear Regression for beans
beanx = np.array(beans['Value']).reshape((-1,1))
beany = np.array(beans['Year']).reshape((-1,1))
beany.reshape((-1,1))
lr = LinearRegression()
regress = lr.fit(beany,beanx)
binter = regress.intercept_ #intercept
binter = binter[0]
bslope = regress.coef_ #slope
bslope = bslope[0]

In [26]:
beans = pd.merge(ci,beans)
aph = beans.groupby(['County'])['Value'].agg(['sum','count']).reset_index()
beans = pd.merge(aph,beans)
beans['Trend fitted yield'] = beans['Year']*bslope+binter
beans['Trend adjusted yield'] = beans['Value']+bslope*(2020-beans['Year'])
beans['Random Price'] = [round(random.uniform(8,17),2) for k in beans.index]
beans['Revenue'] = beans['Random Price']*beans['Trend adjusted yield']
beans['County APH'] = ((beans['sum'])/(beans['count']))

In [38]:
beans['bp guarantee'] = beans['County APH']*beans['Beans Base Price']
beans['hp guarantee'] = beans['County APH']*beans['Beans Harvest Price']
beans['Revenue Guarantee 75%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.75
beans['Revenue Guarantee 80%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.80
beans['Revenue Guarantee 85%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.85
beans['0'] = 0
beans['75% dif'] = beans['Revenue Guarantee 75%']-beans['Revenue']
beans['80% dif'] = beans['Revenue Guarantee 80%']-beans['Revenue']
beans['85% dif'] = beans['Revenue Guarantee 85%']-beans['Revenue']
beans['75% payout'] = beans[['75% dif','0']].max(axis = 1)
beans['80% payout'] = beans[['80% dif','0']].max(axis = 1)
beans['85% payout'] = beans[['85% dif','0']].max(axis = 1)
beans['75% Revenue'] = beans['75% payout']+beans['Revenue']
beans['80% Revenue'] = beans['80% payout']+beans['Revenue']
beans['85% Revenue'] = beans['85% payout']+beans['Revenue']
pd.set_option('display.max_columns', None)
beans

,County,sum,count,Year,Corn Base Price,Corn Harvest Price,Beans Base Price,Beans Harvest Price,Period,State,County ANSI,Commodity,Data Item,Value,CV (%),Trend fitted yield,Trend adjusted yield,Revenue,Random Price,County APH,bp guarantee,hp guarantee,Revenue Guarantee 75%,Revenue Guarantee 80%,Revenue Guarantee 85%,0,75% dif,80% dif,85% dif,75% payout,80% payout,85% payout,75% Revenue,80% Revenue,85% Revenue
0,ADAMS,948.9,20,2001,2.46,2.05,4.67,4.37,YEAR,ILLINOIS,1.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",44.0,NaN,40.011797,61.91491,562.187381,9.08,47.445,221.56815,207.33465,166.176112,177.25452,188.332927,0,-396.011268,-384.932861,-373.854453,0.0,0.0,0.0,562.187381,562.187381,562.187381
1,ADAMS,948.9,20,2002,2.32,2.52,4.50,5.45,YEAR,ILLINOIS,1.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",48.0,NaN,40.954687,64.97202,924.551841,14.23,47.445,213.50250,258.57525,193.931437,206.86020,219.788962,0,-730.620404,-717.691641,-704.762879,0.0,0.0,0.0,924.551841,924.551841,924.551841
2,ADAMS,948.9,20,2003,2.42,2.37,5.26,7.32,YEAR,ILLINOIS,1.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",44.0,NaN,41.897577,60.02913,494.640029,8.24,47.445,249.56070,347.29740,260.473050,277.83792,295.202790,0,-234.166979,-216.802109,-199.437239,0.0,0.0,0.0,494.640029,494.640029,494.640029
3,ADAMS,948.9,20,2004,2.83,1.99,6.72,5.26,YEAR,ILLINOIS,1.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",49.0,NaN,42.840467,64.08624,782.492988,12.21,47.445,318.83040,249.56070,239.122800,255.06432,271.005840,0,-543.370188,-527.428668,-511.487148,0.0,0.0,0.0,782.492988,782.492988,782.492988
4,ADAMS,948.9,20,2005,2.32,1.93,5.53,5.75,YEAR,ILLINOIS,1.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",43.0,NaN,43.783357,57.14335,570.290631,9.98,47.445,262.37085,272.80875,204.606562,218.24700,231.887437,0,-365.684069,-352.043631,-338.403194,0.0,0.0,0.0,570.290631,570.290631,570.290631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,WOODFORD,1118.5,20,2016,3.86,3.49,8.85,9.75,YEAR,ILLINOIS,203.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",67.0,NaN,54.155147,70.77156,961.785500,13.59,55.925,494.93625,545.26875,408.951562,436.21500,463.478437,0,-552.833937,-525.570500,-498.307062,0.0,0.0,0.0,961.785500,961.785500,961.785500
1973,WOODFORD,1118.5,20,2017,3.97,3.49,10.19,9.75,YEAR,ILLINOIS,203.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",60.4,NaN,55.098037,63.22867,544.398848,8.61,55.925,569.87575,545.26875,427.406812,455.90060,484.394387,0,-116.992036,-88.498248,-60.004461,0.0,0.0,0.0,544.398848,544.398848,544.398848
1974,WOODFORD,1118.5,20,2018,3.96,3.68,10.16,8.60,YEAR,ILLINOIS,203.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",64.7,NaN,56.040927,66.58578,789.041493,11.85,55.925,568.19800,480.95500,426.148500,454.55840,482.968300,0,-362.892993,-334.483093,-306.073193,0.0,0.0,0.0,789.041493,789.041493,789.041493
1975,WOODFORD,1118.5,20,2019,4.00,3.90,9.54,9.25,YEAR,ILLINOIS,203.0,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",61.4,NaN,56.983817,62.34289,543.006572,8.71,55.925,533.52450,517.30625,400.143375,426.81960,453.495825,0,-142.863197,-116.186972,-89.510747,0.0,0.0,0.0,543.006572,543.006572,543.006572


In [28]:
aph.reset_index()

,index,County,sum,count
0,0,ADAMS,948.9,20
1,1,ALEXANDER,554.6,14
2,2,BOND,780.7,18
3,3,BOONE,981.6,20
4,4,BROWN,939.5,20
...,...,...,...,...
99,99,WHITESIDE,937.4,18
100,100,WILL,956.1,20
101,101,WILLIAMSON,730.9,19
102,102,WINNEBAGO,953.7,20


In [29]:
beans['bp guarantee'] = beans['County APH']*beans['Beans Base Price']
beans['hp guarantee'] = beans['County APH']*beans['Beans Harvest Price']
beans['Revenue Guarantee 75%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.75
beans['Revenue Guarantee 80%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.80
beans['Revenue Guarantee 85%'] = beans[['bp guarantee','hp guarantee']].max(axis = 1)*.85
beans

,County,sum,count,Year,Corn Base Price,Corn Harvest Price,Beans Base Price,Beans Harvest Price,Period,State,...,0,75% dif,80% dif,85% dif,75% payout,80% payout,85% payout,75% Revenue,80% Revenue,85% Revenue
0,ADAMS,948.9,20,2001,2.46,2.05,4.67,4.37,YEAR,ILLINOIS,...,0,-396.011268,-384.932861,-373.854453,0.0,0.0,0.0,562.187381,562.187381,562.187381
1,ADAMS,948.9,20,2002,2.32,2.52,4.50,5.45,YEAR,ILLINOIS,...,0,-730.620404,-717.691641,-704.762879,0.0,0.0,0.0,924.551841,924.551841,924.551841
2,ADAMS,948.9,20,2003,2.42,2.37,5.26,7.32,YEAR,ILLINOIS,...,0,-234.166979,-216.802109,-199.437239,0.0,0.0,0.0,494.640029,494.640029,494.640029
3,ADAMS,948.9,20,2004,2.83,1.99,6.72,5.26,YEAR,ILLINOIS,...,0,-543.370188,-527.428668,-511.487148,0.0,0.0,0.0,782.492988,782.492988,782.492988
4,ADAMS,948.9,20,2005,2.32,1.93,5.53,5.75,YEAR,ILLINOIS,...,0,-365.684069,-352.043631,-338.403194,0.0,0.0,0.0,570.290631,570.290631,570.290631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,WOODFORD,1118.5,20,2016,3.86,3.49,8.85,9.75,YEAR,ILLINOIS,...,0,-552.833937,-525.570500,-498.307062,0.0,0.0,0.0,961.785500,961.785500,961.785500
1973,WOODFORD,1118.5,20,2017,3.97,3.49,10.19,9.75,YEAR,ILLINOIS,...,0,-116.992036,-88.498248,-60.004461,0.0,0.0,0.0,544.398848,544.398848,544.398848
1974,WOODFORD,1118.5,20,2018,3.96,3.68,10.16,8.60,YEAR,ILLINOIS,...,0,-362.892993,-334.483093,-306.073193,0.0,0.0,0.0,789.041493,789.041493,789.041493
1975,WOODFORD,1118.5,20,2019,4.00,3.90,9.54,9.25,YEAR,ILLINOIS,...,0,-142.863197,-116.186972,-89.510747,0.0,0.0,0.0,543.006572,543.006572,543.006572


In [30]:
aph = beans.groupby(['County'])['Value'].agg(['sum','count'])
aph

,sum,count
County,,
ADAMS,948.9,20
ALEXANDER,554.6,14
BOND,780.7,18
BOONE,981.6,20
BROWN,939.5,20
...,...,...
WHITESIDE,937.4,18
WILL,956.1,20
WILLIAMSON,730.9,19


In [31]:
beans['County APH']*beans['Beans Base Price']

0       221.56815
1       213.50250
2       249.56070
3       318.83040
4       262.37085
          ...    
1972    494.93625
1973    569.87575
1974    568.19800
1975    533.52450
1976    512.83225
Length: 1977, dtype: float64

In [32]:
p_data = pd.read_csv('/Users/isaacwerries/Desktop/Price Data.csv')
del p_data['Program'], p_data['Week Ending'], p_data['Geo Level'], p_data['State ANSI'], p_data['County ANSI'], p_data['Ag District'], p_data['Ag District Code'], p_data['County'],p_data['Zip Code'], p_data['Region'], p_data['watershed_code'], p_data['Watershed'], p_data['Domain'], p_data['Domain Category'], p_data['CV (%)']
corn_p = p_data[p_data['Commodity'] == 'CORN'] #splitting the dataframe based on crop
corn_p = corn_p[corn_p['Year'] > 2000].copy()
bean_p = p_data[p_data['Commodity'] == 'SOYBEANS'].copy() #splitting the dataframe based on crop
bean_p = bean_p[bean_p['Year'] > 2000].copy()
corn_p

,Year,Period,State,Commodity,Data Item,Value
0,2021,JAN,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",4.10
2,2021,FEB,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",4.69
4,2021,MAR,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",4.96
6,2020,JAN,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",3.76
8,2020,FEB,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",3.71
...,...,...,...,...,...,...
476,2001,AUG,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",1.99
478,2001,SEP,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",1.94
480,2001,OCT,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",1.85
482,2001,NOV,ILLINOIS,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",1.91


In [37]:
fig = px.line(beans, x="Year", y=["85% Revenue","Revenue"], animation_frame="County", 
           log_x=True, range_x=[2000,2021], range_y=[300,1300])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()
